### Lets import all necessary libraries ###

In [8]:
import os
import streamlit as st
from PyPDF2 import PdfReader
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
from langchain.chains.question_answering import load_qa_chain

#### Function to extract text from PDF ####

In [9]:
def extract_text_from_pdf(pdf_path) :
    reader = PdfReader ( pdf_path)
    text = " "
    for page in reader.pages:
        text += page .extract_text()
    return text

#### Create FAISS vector store ####

In [10]:
def faiss_store(text, path="false_index") :
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
    chunks = splitter.split_text(text)
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    vector_store.save_local(path)
    


#### Load FAISS vector store ####

In [ ]:
def load_faiss_vector_store(path="faiss_index") :
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.load_local(path,embeddings,allow_dangerous_deserialization=True)
    return vector_store

# BUild QA Chain 
def build_qa_chain(vector_store_path="faiss_index") :
    vector_store = load_faiss_vector_store(vector_store_path)
    retriver = vector_store.as_retriever()
    # Load QA chain for combining our documents
    llm = Ollama(model="llama3.2")
    qa_chain = load_qa_chain(llm, chain_type="stuff")
    qa_chain = RetrievalQA(retriver=retriver,combine_document_chain=qa_chain)
    return qa_chain


### Now comes the Streamlit APP part 

In [ ]:
st.title("RAG Chatbot with FAISS and LLaMA")
st.write("Upload a PDF and ask questions based on its content.")
uploaded_file = st.file_uploader("Upload your PDF file", type="pdf")
if uploaded_file is not None:
    pdf_path = f"uploaded/{uploaded_file.name}"
    os.makedirs("uploaded", exist_ok=True)
    with open(pdf_path, "wb") as f:
        f.write(uploaded_file.getbuffer())
    text = extract_text_from_pdf(pdf_path)
    st.info("Creating FAISS vector store...")
    faiss_store(text)
    st.info("Initializing chatbot...")
    qa_chain = build_qa_chain()
    st.success("Chatbot is ready!")


if 'qa_chain' in locals():
    question = st.text_input("Ask a question about the uploaded PDF:")
    if question:
        st.info("Querying the document...")
        answer = qa_chain.run(question)
        st.success(f"Answer: {answer}")
